In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
from IPython.display import SVG
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D, Input
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.utils import to_categorical, model_to_dot, plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau

In [2]:
train_data_dir = "C:\\Users\\Shabnoor\\OneDrive\\Desktop\\breed\\images\\Images\\"

# Selected 10 Dog Breeds
selected_breeds = [
    'n02085936-Maltese_dog', 'n02086646-Blenheim_spaniel', 'n02086910-papillon', 
    'n02087046-toy_terrier', 'n02087394-Rhodesian_ridgeback', 'n02088094-Afghan_hound',
    'n02088238-basset', 'n02088364-beagle', 'n02088466-bloodhound', 'n02088632-bluetick'
]

# Image Dimensions
img_width, img_height = 299, 299
channels = 3
batch_size = 64

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2,
)

valid_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
)

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    classes=selected_breeds,
    subset='training',
    shuffle=True,
    seed=1337
)

valid_generator = valid_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    classes=selected_breeds,
    subset='validation',
    shuffle=True,
    seed=1337
)

Found 1561 images belonging to 10 classes.
Found 386 images belonging to 10 classes.


In [5]:
num_classes = len(train_generator.class_indices)  
train_labels = train_generator.classes 
train_labels = to_categorical(train_labels, num_classes=num_classes)
valid_labels = valid_generator.classes 
valid_labels = to_categorical(valid_labels, num_classes=num_classes)
nb_train_samples = len(train_generator.filenames)  
nb_valid_samples = len(valid_generator.filenames)

In [6]:
InceptionV3 = applications.InceptionV3(include_top= False, input_shape= (img_width, img_height, channels), weights= 'imagenet')
InceptionV3.summary()

Model: "inception_v3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 147, 147,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 73, 73,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 73, 73,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 73, 73,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 73, 73,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 71, 71,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 71, 71,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 71, 71,    │          0 │ batch_normalizat

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 21,768,352 (83.04 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [7]:
model = Sequential()

for layer in InceptionV3.layers:
    layer.trainable= False

model.add(InceptionV3)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))  # Change the number of units to 10

In [9]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='auto'
)

earlystop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=1,
    mode='auto',
    restore_best_weights=True
)

csvlogger = CSVLogger(
    filename="training_log.csv",
    separator=",",
    append=False
)

reduceLR = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    verbose=1,
    mode='auto'
)

callbacks = [checkpoint, earlystop, csvlogger,reduceLR]
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Use learning_rate instead of lr
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [11]:
# Training the Model
history = model.fit(
    train_generator, 
    epochs=15,
    steps_per_epoch=nb_train_samples // batch_size,
    validation_data=valid_generator, 
    validation_steps=nb_valid_samples // batch_size ,
    verbose=2, 
    callbacks=callbacks,
    shuffle=True
)


Epoch 1/15


c:\Users\Shabnoor\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_loss improved from inf to 1.90641, saving model to best_model.keras
24/24 - 335s - 14s/step - accuracy: 0.1725 - loss: 2.1800 - val_accuracy: 0.4427 - val_loss: 1.9064 - learning_rate: 1.0000e-04
Epoch 2/15


c:\Users\Shabnoor\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_loss improved from 1.90641 to 1.72323, saving model to best_model.keras
24/24 - 8s - 329ms/step - accuracy: 0.2400 - loss: 1.9852 - val_accuracy: 1.0000 - val_loss: 1.7232 - learning_rate: 1.0000e-04
Epoch 3/15

Epoch 3: val_loss improved from 1.72323 to 1.50885, saving model to best_model.keras
24/24 - 281s - 12s/step - accuracy: 0.5217 - loss: 1.7496 - val_accuracy: 0.8255 - val_loss: 1.5088 - learning_rate: 1.0000e-04
Epoch 4/15

Epoch 4: val_loss improved from 1.50885 to 1.07289, saving model to best_model.keras
24/24 - 15s - 613ms/step - accuracy: 0.7344 - loss: 1.5597 - val_accuracy: 1.0000 - val_loss: 1.0729 - learning_rate: 1.0000e-04
Epoch 5/15

Epoch 5: val_loss did not improve from 1.07289
24/24 - 237s - 10s/step - accuracy: 0.7682 - loss: 1.3830 - val_accuracy: 0.9375 - val_loss: 1.1817 - learning_rate: 1.0000e-04
Epoch 6/15

Epoch 6: val_loss improved from 1.07289 to 0.83533, saving model to best_model.keras
24/24 - 9s - 384ms/step - accuracy: 0.8125 - loss: 

Evaluation

In [12]:
(eval_loss, eval_accuracy) = model.evaluate(valid_generator, batch_size=batch_size, verbose=1)
print('Validation Loss:', eval_loss)
print('Validation Accuracy:', eval_accuracy)

7/7 ━━━━━━━━━━━━━━━━━━━━ 46s 6s/step - accuracy: 0.9787 - loss: 0.4305
Validation Loss: 0.41443684697151184
Validation Accuracy: 0.984455943107605
